# Pair Programming 26/05 Test Estadísticos III - Tatiana y Guada

In [ ]:
pip install researchpy

In [ ]:
!pip install scipy

In [4]:
import pandas as pd
import random 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene


plt.rcParams["figure.figsize"] = (10,8) 

Usaremos el dataset de listings_limpio-2 en el caso que no hayamos llegado a terminar el pair de ayer. Si tenéis el csv de ayer es preferido que uséis ese.

# Ejercicio 1
Chequead los outliers y haced la gestión de ellos, como aprendimos en las lecciones de limpieza.

# Ejercicio 2
¿Sigue la columma price una distribución normal? Chequeadlo visual y analiticamente.
En caso de que no siga una distribución normal, realizad la transformación necesaria para que se ajuste a una normal.

# Ejercicio 3
Chequead la independencia entre las siguientes columnas: neighbourhood_group y room_type

# Ejercicio 4
Chequead la heterocedasticidad de la columna room_type frente al price